In [ ]:
!pip install scipy

In [1]:
import json, requests, time
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, kruskal

engine = create_engine('postgresql://postgres:argmax@pg:5432/postgres')

# Data
Every time a user opens a mobile app, an auction is going on behind the scenes. The highest bidder gets to advertise his ad to the user.
## Auctions Table

In [139]:
sql_query = 'SELECT * FROM auctions;'
with engine.connect() as db_con:
    df = pd.read_sql(sql_query, con=db_con)

df

,id,eventTimestamp,unitDisplayType,brandName,bundleId,countryCode,deviceId,osAndVersion,bidFloorPrice,sentPrice
0,1,1657758857892,rewarded,Generic,com.loop.match3d,US,6a0a94554cf,Android-4.0,0.05,0.06
1,2,1657210707978,interstitial,Generic,com.loop.match3d,ZA,6a0b0e59f45,Android-4.0,0.01,0.16
2,3,1657392939412,interstitial,Motorola,com.YayySAL.DodgeAgent,BR,6a0fa820c46,Android-10.0,0.05,0.06
3,4,1657386816882,interstitial,Huawei,com.tintash.nailsalon,IQ,6a142bdbea2,Android-9.0,0.01,0.05
4,5,1657211600823,banner,Generic,com.tilegarden.match3,US,6a16943a771,Android-4.0,0.01,0.03
...,...,...,...,...,...,...,...,...,...,...
41579,41580,1657731605063,banner,Generic,com.tilegarden.match3,NZ,80edf5837e8,Android-4.0,0.01,0.02
41580,41581,1657657131101,interstitial,Apple,1436213906,VN,80ee6a5eebf,iOS-13.6,0.01,0.90
41581,41582,1657156070854,rewarded,Generic,com.loop.match3d,US,80efe531afc,Android-4.0,0.05,0.69
41582,41583,1657659603568,banner,Apple,1502447854,US,80f02dd397f,iOS-15.0,0.01,0.10


## App Vectors table
We've gathered the first few sentences from the app store description and embedded it with a [model](https://huggingface.co/mixedbread-ai/mxbai-embed-large-v1)

In [192]:
sql_query = f'''
SELECT
    *
FROM app_vectors
'''
has_embedding = False
while not has_embedding:
    with engine.connect() as db_con:
        df_embedding = pd.read_sql(sql_query, con=db_con)
    has_embedding = (~df_embedding["embedding"].isna()).all()
    if not has_embedding:
        print("Waiting for embeddings...")
        time.sleep(15)

df_embedding

,id,bundleId,content,embedding
0,1,com.loop.match3d,"Get ready for a new, challenging and original ...","[0.3168352,0.6250086,-0.12035585,0.21742323,-0..."
1,2,com.YayySAL.DodgeAgent,"Your mission, should you choose to accept it, ...","[0.5411282,0.7446755,-0.5817451,0.95967793,0.0..."
2,3,com.tintash.nailsalon,It is manicure madness over here and it�s your...,"[0.31176656,0.6059464,-1.1395648,0.07631045,-0..."
3,4,com.tilegarden.match3,\nIf you enjoy playing Match 3 tile & mahjong ...,"[0.050924458,1.0967327,-0.30831614,0.13220672,..."
4,6,com.AppIdeas.LevelUpRunner,"Are you a real hero? Are you the strongest, th...","[0.897561,0.46442518,-0.18019226,1.2617054,0.0..."
5,17,1582745578,Dominate the coal industry and become the next...,"[0.88490486,0.65851957,-0.3386694,0.18323663,0..."
6,18,1569586264,Choose the appropriate outfit to make it throu...,"[0.2000534,0.36083463,-0.13792263,0.34308642,-..."
7,5,com.kamilbilge.ropesavior3d,Become the hero we need in this tangled puzzle...,"[0.5112473,0.5128061,-0.05630451,1.2819325,0.1..."
8,7,se.ace.fishinc,Gather your riches and travel across the open ...,"[0.73094344,0.4112171,-0.6196014,0.53501374,0...."
9,8,com.volt.dresstoimpress,\nChoose the appropriate outfit to make it thr...,"[0.2000534,0.36083463,-0.13792263,0.34308642,-..."


We can use the `<=>` operator to run vector search within the database

In [141]:

vec = json.loads(df_embedding.embedding[0]) # get the first embedding
print ("Embedding size: {l}".format(l=len(vec)))

sql_query = f'''
SELECT
    "bundleId"
FROM app_vectors
ORDER BY embedding<=>'{json.dumps(vec)}'
'''
with engine.connect() as db_con:
    df_vector = pd.read_sql(sql_query, con=db_con)

df_vector

Embedding size: 1024


,bundleId
0,com.loop.match3d
1,1502447854
2,1529614832
3,com.tilegarden.match3
4,com.kamilbilge.ropesavior3d
5,1579489488
6,1436213906
7,1586795332
8,com.AppIdeas.LevelUpRunner
9,se.ace.fishinc


# What you need to do
## The hypothesis
We assume that apps with similar desciptions, would have a similar asking price in the auctions (`sentPrice` column).

Use cosine similarity (`<=>`) on the embeddings to find similar apps, and any statistical tools you find suitable to prove or disprove this hypothesis.

## Is it consistent?
There are several other features in the auctions table (such as `CountryCode` and `OS`), 
Do your findings hold for those as well?

## The analysis

Let's denote:

* $\mu_i$ : Mean price of app $i$.
* $\mu_{similae,i}$ : Mean price of the most similar app (according to cosine similarity) to app $$
i.

**Null hypothesis (H0):**

$H_0 : \mu_i = \mu_{similae,i}$

The null hypothesis states that there is no significant difference in mean prices between app $$
i and its most similar app.

**Alternative Hypothesis (H1):**

$H_1 : \mu_i \neq \mu_{similae,i}$

The alternative hypothesis suggests that there is a significant difference in mean prices between app $$
i and its most similar app.and its most similar app.

**Statistical test:**

To test this hypothesis, I used Welch's T-Test (unequal variance T-Test). This test is appropriate when comparing the means of two groups with potentially unequal variances and sample sizes. It's robust and suitable for datasets where assumptions of equal variances or normality might not hold.

In [193]:
# Function that retrieves the most similar apps based on their embeddings using cosine similarity.
def find_similar_apps(x):
    vec = json.loads(x)
    sql_query = f'''
    SELECT
        "bundleId"
    FROM app_vectors
    ORDER BY embedding<=>'{json.dumps(vec)}'
    '''
    with engine.connect() as db_con:
        df_vector = pd.read_sql(sql_query, con=db_con)

    return df_vector.values.flatten()

# Apply the find_similar_apps function and store the results in a new column 'similarApps'
df_embedding["similarApps"] = df_embedding.embedding.apply(lambda x: find_similar_apps(x))
df_embedding

,id,bundleId,content,embedding,similarApps
0,1,com.loop.match3d,"Get ready for a new, challenging and original ...","[0.3168352,0.6250086,-0.12035585,0.21742323,-0...","[com.loop.match3d, 1502447854, 1529614832, com..."
1,2,com.YayySAL.DodgeAgent,"Your mission, should you choose to accept it, ...","[0.5411282,0.7446755,-0.5817451,0.95967793,0.0...","[1542256628, com.YayySAL.DodgeAgent, com.kamil..."
2,3,com.tintash.nailsalon,It is manicure madness over here and it�s your...,"[0.31176656,0.6059464,-1.1395648,0.07631045,-0...","[1523081624, com.tintash.nailsalon, 1582745578..."
3,4,com.tilegarden.match3,\nIf you enjoy playing Match 3 tile & mahjong ...,"[0.050924458,1.0967327,-0.30831614,0.13220672,...","[com.tilegarden.match3, 1529614832, 1502447854..."
4,6,com.AppIdeas.LevelUpRunner,"Are you a real hero? Are you the strongest, th...","[0.897561,0.46442518,-0.18019226,1.2617054,0.0...","[1586795332, com.AppIdeas.LevelUpRunner, com.k..."
5,17,1582745578,Dominate the coal industry and become the next...,"[0.88490486,0.65851957,-0.3386694,0.18323663,0...","[1582745578, dovi.coalmining.inc, 1436213906, ..."
6,18,1569586264,Choose the appropriate outfit to make it throu...,"[0.2000534,0.36083463,-0.13792263,0.34308642,-...","[1569586264, com.volt.dresstoimpress, 15867953..."
7,5,com.kamilbilge.ropesavior3d,Become the hero we need in this tangled puzzle...,"[0.5112473,0.5128061,-0.05630451,1.2819325,0.1...","[1579489488, com.kamilbilge.ropesavior3d, 1542..."
8,7,se.ace.fishinc,Gather your riches and travel across the open ...,"[0.73094344,0.4112171,-0.6196014,0.53501374,0....","[se.ace.fishinc, 1436213906, 1582745578, dovi...."
9,8,com.volt.dresstoimpress,\nChoose the appropriate outfit to make it thr...,"[0.2000534,0.36083463,-0.13792263,0.34308642,-...","[1569586264, com.volt.dresstoimpress, 15867953..."


In [194]:
# Store all prices of each app in a new column 'sentPrices'
prices = df.groupby("bundleId")["sentPrice"].apply(lambda x: x.values)

df_embedding = df_embedding.merge(prices, how='inner', on='bundleId')
df_embedding = df_embedding.rename(columns={'sentPrice': 'sentPrices'}) 
df_embedding

,id,bundleId,content,embedding,similarApps,sentPrices
0,1,com.loop.match3d,"Get ready for a new, challenging and original ...","[0.3168352,0.6250086,-0.12035585,0.21742323,-0...","[com.loop.match3d, 1502447854, 1529614832, com...","[0.06, 0.16, 0.24, 0.02, 0.3, 0.03, 0.03, 0.03..."
1,2,com.YayySAL.DodgeAgent,"Your mission, should you choose to accept it, ...","[0.5411282,0.7446755,-0.5817451,0.95967793,0.0...","[1542256628, com.YayySAL.DodgeAgent, com.kamil...","[0.06, 0.09, 0.22, 0.05, 0.06, 0.11, 0.79, 0.0..."
2,3,com.tintash.nailsalon,It is manicure madness over here and it�s your...,"[0.31176656,0.6059464,-1.1395648,0.07631045,-0...","[1523081624, com.tintash.nailsalon, 1582745578...","[0.05, 0.04, 0.03, 0.04, 0.06, 0.06, 9.35, 0.2..."
3,4,com.tilegarden.match3,\nIf you enjoy playing Match 3 tile & mahjong ...,"[0.050924458,1.0967327,-0.30831614,0.13220672,...","[com.tilegarden.match3, 1529614832, 1502447854...","[0.03, 4.42, 0.2, 0.04, 6.97, 0.02, 6.59, 1.08..."
4,6,com.AppIdeas.LevelUpRunner,"Are you a real hero? Are you the strongest, th...","[0.897561,0.46442518,-0.18019226,1.2617054,0.0...","[1586795332, com.AppIdeas.LevelUpRunner, com.k...","[0.19, 0.47, 0.07, 0.05, 0.07, 0.05, 0.08, 0.0..."
5,17,1582745578,Dominate the coal industry and become the next...,"[0.88490486,0.65851957,-0.3386694,0.18323663,0...","[1582745578, dovi.coalmining.inc, 1436213906, ...","[0.05, 1.42, 0.14, 0.44, 1.0, 0.05, 0.1, 0.06,..."
6,18,1569586264,Choose the appropriate outfit to make it throu...,"[0.2000534,0.36083463,-0.13792263,0.34308642,-...","[1569586264, com.volt.dresstoimpress, 15867953...","[7.81, 29.81, 1.23, 9.16, 42.86, 7.15, 9.08, 8..."
7,5,com.kamilbilge.ropesavior3d,Become the hero we need in this tangled puzzle...,"[0.5112473,0.5128061,-0.05630451,1.2819325,0.1...","[1579489488, com.kamilbilge.ropesavior3d, 1542...","[0.07, 0.06, 0.2, 0.05, 0.05, 0.06, 0.74, 0.09..."
8,7,se.ace.fishinc,Gather your riches and travel across the open ...,"[0.73094344,0.4112171,-0.6196014,0.53501374,0....","[se.ace.fishinc, 1436213906, 1582745578, dovi....","[0.02, 0.02, 0.81, 0.03, 0.02, 0.81, 0.02, 0.0..."
9,8,com.volt.dresstoimpress,\nChoose the appropriate outfit to make it thr...,"[0.2000534,0.36083463,-0.13792263,0.34308642,-...","[1569586264, com.volt.dresstoimpress, 15867953...","[0.01, 0.07, 0.04, 0.01, 0.05, 0.61, 1.28, 0.0..."


In [195]:
# Function to calculate mean price difference between similar apps within each row
def check_price_similarity(row):
    similar_apps = row['similarApps']
    prices = row['sentPrices']
    p_values = []
    for app in similar_apps:
        app_prices = df_embedding[df_embedding['bundleId'] == app]['sentPrices'].values[0]
        # Perform Welch's t-test for independent samples
        t_statistic, p_value = ttest_ind(prices, app_prices, equal_var=False)
        p_values.append(p_value)
    return p_values

In [198]:
# Store all p-values of statistical test in a new column 'p_values'
df_embedding['p_values'] = df_embedding.apply(lambda row: check_price_similarity(row), axis=1) 
df_embedding

,id,bundleId,content,embedding,similarApps,sentPrices,p_values
0,1,com.loop.match3d,"Get ready for a new, challenging and original ...","[0.3168352,0.6250086,-0.12035585,0.21742323,-0...","[com.loop.match3d, 1502447854, 1529614832, com...","[0.06, 0.16, 0.24, 0.02, 0.3, 0.03, 0.03, 0.03...","[1.0, 4.815677350205852e-05, 0.010638827849438..."
1,2,com.YayySAL.DodgeAgent,"Your mission, should you choose to accept it, ...","[0.5411282,0.7446755,-0.5817451,0.95967793,0.0...","[1542256628, com.YayySAL.DodgeAgent, com.kamil...","[0.06, 0.09, 0.22, 0.05, 0.06, 0.11, 0.79, 0.0...","[4.160416285214159e-07, 1.0, 0.911142231832393..."
2,3,com.tintash.nailsalon,It is manicure madness over here and it�s your...,"[0.31176656,0.6059464,-1.1395648,0.07631045,-0...","[1523081624, com.tintash.nailsalon, 1582745578...","[0.05, 0.04, 0.03, 0.04, 0.06, 0.06, 9.35, 0.2...","[1.5655556314746647e-18, 1.0, 0.12043152857363..."
3,4,com.tilegarden.match3,\nIf you enjoy playing Match 3 tile & mahjong ...,"[0.050924458,1.0967327,-0.30831614,0.13220672,...","[com.tilegarden.match3, 1529614832, 1502447854...","[0.03, 4.42, 0.2, 0.04, 6.97, 0.02, 6.59, 1.08...","[1.0, 0.22694064863045638, 0.9154121614869442,..."
4,6,com.AppIdeas.LevelUpRunner,"Are you a real hero? Are you the strongest, th...","[0.897561,0.46442518,-0.18019226,1.2617054,0.0...","[1586795332, com.AppIdeas.LevelUpRunner, com.k...","[0.19, 0.47, 0.07, 0.05, 0.07, 0.05, 0.08, 0.0...","[5.344443041550169e-08, 1.0, 6.228070643553652..."
5,17,1582745578,Dominate the coal industry and become the next...,"[0.88490486,0.65851957,-0.3386694,0.18323663,0...","[1582745578, dovi.coalmining.inc, 1436213906, ...","[0.05, 1.42, 0.14, 0.44, 1.0, 0.05, 0.1, 0.06,...","[1.0, 0.1902659813799068, 0.048242344178445126..."
6,18,1569586264,Choose the appropriate outfit to make it throu...,"[0.2000534,0.36083463,-0.13792263,0.34308642,-...","[1569586264, com.volt.dresstoimpress, 15867953...","[7.81, 29.81, 1.23, 9.16, 42.86, 7.15, 9.08, 8...","[1.0, 3.1835471698847754e-11, 2.28538181429291..."
7,5,com.kamilbilge.ropesavior3d,Become the hero we need in this tangled puzzle...,"[0.5112473,0.5128061,-0.05630451,1.2819325,0.1...","[1579489488, com.kamilbilge.ropesavior3d, 1542...","[0.07, 0.06, 0.2, 0.05, 0.05, 0.06, 0.74, 0.09...","[0.0015978353142641739, 1.0, 4.167811322168410..."
8,7,se.ace.fishinc,Gather your riches and travel across the open ...,"[0.73094344,0.4112171,-0.6196014,0.53501374,0....","[se.ace.fishinc, 1436213906, 1582745578, dovi....","[0.02, 0.02, 0.81, 0.03, 0.02, 0.81, 0.02, 0.0...","[1.0, 3.104332362434945e-11, 0.772325557960628..."
9,8,com.volt.dresstoimpress,\nChoose the appropriate outfit to make it thr...,"[0.2000534,0.36083463,-0.13792263,0.34308642,-...","[1569586264, com.volt.dresstoimpress, 15867953...","[0.01, 0.07, 0.04, 0.01, 0.05, 0.61, 1.28, 0.0...","[3.1835471698847754e-11, 1.0, 1.74537852624045..."


In [214]:
# Create a new DataFrame with 'bundleId' as the index and p-values as columns
df_pvalues = pd.DataFrame(df_embedding['p_values'].tolist(), index=df_embedding['bundleId'])

# Generate new column names
new_column_names = {i: f'similarApp{i+1}' for i in range(18)}  # Generate new column names dynamically

# Rename the columns with the new names
df_pvalues.rename(columns=new_column_names, inplace=True)

df_pvalues

,similarApp1,similarApp2,similarApp3,similarApp4,similarApp5,similarApp6,similarApp7,similarApp8,similarApp9,similarApp10,similarApp11,similarApp12,similarApp13,similarApp14,similarApp15,similarApp16,similarApp17,similarApp18
bundleId,,,,,,,,,,,,,,,,,,
com.loop.match3d,1.000000e+00,4.815677e-05,1.063883e-02,8.201788e-06,8.915730e-28,1.647076e-01,2.876383e-14,5.215385e-01,3.155779e-54,9.338884e-04,4.015048e-30,1.643404e-10,1.296382e-25,3.572248e-02,5.277610e-01,4.879482e-09,6.552902e-03,2.377549e-35
com.YayySAL.DodgeAgent,4.160416e-07,1.000000e+00,9.111422e-01,1.596723e-03,4.868162e-13,2.082020e-05,3.989230e-23,2.584653e-20,1.513284e-01,2.241864e-01,2.266079e-41,1.296382e-25,1.207787e-23,5.854044e-05,9.954212e-01,3.185852e-11,2.881690e-12,9.330191e-03
com.tintash.nailsalon,1.565556e-18,1.000000e+00,1.204315e-01,1.417772e-02,2.497082e-11,5.397524e-04,1.142959e-23,1.288336e-24,2.377549e-35,3.503214e-06,1.627281e-06,1.541891e-52,1.482015e-27,2.284822e-05,1.189601e-04,1.672632e-03,9.330191e-03,4.018127e-08
com.tilegarden.match3,1.000000e+00,2.269406e-01,9.154122e-01,8.201788e-06,5.107139e-24,1.480711e-05,1.327989e-07,2.050144e-02,1.544429e-24,7.325292e-10,6.171454e-01,1.207787e-23,9.221359e-01,1.042772e-15,1.482015e-27,1.313634e-09,4.032275e-33,2.854614e-02
com.AppIdeas.LevelUpRunner,5.344443e-08,1.000000e+00,6.228071e-23,2.922377e-06,4.868162e-13,2.020186e-09,4.695366e-44,1.883730e-11,2.196473e-28,3.155779e-54,1.758101e-26,3.503214e-06,1.628764e-30,9.113747e-02,5.602328e-05,7.176460e-06,1.529324e-72,4.032275e-33
1582745578,1.000000e+00,1.902660e-01,4.824234e-02,7.723256e-01,3.706100e-01,1.204315e-01,3.812161e-01,1.526051e-01,9.405000e-01,1.513284e-01,9.585535e-01,5.277610e-01,6.356242e-01,9.113747e-02,9.221359e-01,4.899796e-01,1.510824e-01,3.345720e-09
1569586264,1.000000e+00,3.183547e-11,2.285382e-10,1.883730e-11,7.009247e-10,2.497082e-11,9.603302e-11,1.643404e-10,4.238922e-09,7.325292e-10,3.217316e-11,9.916489e-11,3.185852e-11,5.609786e-10,5.072692e-08,3.291343e-10,3.345720e-09,4.091704e-11
com.kamilbilge.ropesavior3d,1.597835e-03,1.000000e+00,4.167811e-07,9.111422e-01,2.073965e-05,6.228071e-23,1.407935e-20,8.915730e-28,2.302829e-01,1.526051e-01,5.986414e-05,3.735438e-23,5.107139e-24,2.872871e-44,3.217316e-11,8.714224e-01,3.924337e-13,1.672632e-03
se.ace.fishinc,1.000000e+00,3.104332e-11,7.723256e-01,7.063734e-17,3.986084e-02,9.338884e-04,2.266079e-41,3.942859e-01,2.050144e-02,1.523349e-03,2.872871e-44,1.541891e-52,4.971447e-09,4.046272e-01,1.529324e-72,5.284451e-02,3.640650e-47,3.291343e-10


In [215]:
# Significance level
alpha = 0.05

# Function to check if p-value is rejected
def check_reject(p_value):
    # Both apps are the same app
    if p_value == 1:
        return "same app"
    # Reject the null hypothesis 
    elif p_value < alpha:
        return "reject"
    # Fail to reject the null hypothesis
    else:
        return "fail"

# Apply the function to each value in the DataFrame
df_status = df_pvalues.map(check_reject)

# Print the DataFrame with rejection status
df_status

,similarApp1,similarApp2,similarApp3,similarApp4,similarApp5,similarApp6,similarApp7,similarApp8,similarApp9,similarApp10,similarApp11,similarApp12,similarApp13,similarApp14,similarApp15,similarApp16,similarApp17,similarApp18
bundleId,,,,,,,,,,,,,,,,,,
com.loop.match3d,same app,reject,reject,reject,reject,fail,reject,fail,reject,reject,reject,reject,reject,reject,fail,reject,reject,reject
com.YayySAL.DodgeAgent,reject,same app,fail,reject,reject,reject,reject,reject,fail,fail,reject,reject,reject,reject,fail,reject,reject,reject
com.tintash.nailsalon,reject,same app,fail,reject,reject,reject,reject,reject,reject,reject,reject,reject,reject,reject,reject,reject,reject,reject
com.tilegarden.match3,same app,fail,fail,reject,reject,reject,reject,reject,reject,reject,fail,reject,fail,reject,reject,reject,reject,reject
com.AppIdeas.LevelUpRunner,reject,same app,reject,reject,reject,reject,reject,reject,reject,reject,reject,reject,reject,fail,reject,reject,reject,reject
1582745578,same app,fail,reject,fail,fail,fail,fail,fail,fail,fail,fail,fail,fail,fail,fail,fail,fail,reject
1569586264,same app,reject,reject,reject,reject,reject,reject,reject,reject,reject,reject,reject,reject,reject,reject,reject,reject,reject
com.kamilbilge.ropesavior3d,reject,same app,reject,fail,reject,reject,reject,reject,fail,fail,reject,reject,reject,reject,reject,fail,reject,reject
se.ace.fishinc,same app,reject,fail,reject,reject,reject,reject,fail,reject,reject,reject,reject,reject,fail,reject,fail,reject,reject


In [255]:
# Function to check frequency based on 'similarApp1' and 'similarApp2' columns
def check_frequency(row):
    # If both apps are the same, we take the second app in the list
    if row['similarApp1'] == 'same app':
        return row['similarApp2']
    else:
        return row['similarApp1']

# Apply the function to each row and count the occurrences
frequency_counts = df_status.apply(check_frequency, axis=1).value_counts()

# Print the frequency of reject and fail outcomes
print("Frequency of Hypothesis Test Results:")
print(frequency_counts)

# Calculate the percentage of 'fail' entries and 'reject' enteries
percentage_reject = (frequency_counts.get('fail', 0) / frequency_counts.sum()) * 100
percentage_fail = (frequency_counts.get('fail', 0) / frequency_counts.sum()) * 100
print(f"Percentage of 'fail': {round(percentage_fail,2)}%")
print(f"Percentage of 'reject': {100-round(percentage_fail,2)}%")

# Determine the overall conclusion based on the majority outcome
if frequency_counts.get('reject', 0) > frequency_counts.get('fail', 0):
    print("\nThe hypothesis is consistent across the dataset. There is a significant difference in prices between apps and their similar apps.")
else:
    print("\nThe hypothesis is not consistent across the dataset. There is no significant difference in prices between apps and their similar apps.")

Frequency of Hypothesis Test Results:
reject    14
fail       4
Name: count, dtype: int64
Percentage of 'fail': 22.22%
Percentage of 'reject': 77.78%

The hypothesis is consistent across the dataset. There is a significant difference in prices between apps and their similar apps.


**The results:**
* We conducted Welch's T-Test for each app and its most similar app (according cosine similarity)
* For each pair of similar apps, we calculated the p-value from the T-Test. The p-value represents the probability of observing the data if the null hypothesis (no difference in prices) were true.
* If the p-value is less than a chosen significance level (e.g., 0.05), we **reject** the null hypothesis and conclude that there is a significant difference in prices between similar apps. This would support the alternative hypothesis.
* If the p-value is greater than the significance level, we **fail** to reject the null hypothesis, indicating that there is no significant difference in prices between similar apps.

**Based on these results:**
* If the majority of p-values are less than the significance level, we would reject the null hypothesis for most apps and conclude that similar apps tend to have similar prices.
* If the majority of p-values are greater than the significance level, we would fail to reject the null hypothesis for most apps and conclude that there is no significant relationship between app descriptions and prices.

**Conclusions:**

Based on the statistical analysis, I found that the percentage of tests failing to reject the null hypothesis is 22.22%, while the percentage of tests rejecting the null hypothesis is 77.78%. This inconsistency indicates that the hypothesis is not supported by the data.

The majority of apps (77.78%) showed significant differences in prices between similar apps, suggesting that app descriptions alone may not be a reliable predictor of pricing in app auctions. Further investigation and refinement of the hypothesis are recommended to establish a stronger relationship between app descriptions and prices in auctions.

**Check consistent:**

Now I will check if my findings also hold for other features (such as `CountryCode` and `OS`) as well.

`CountryCode`

**Null hypothesis (H0):** The mean prices of applications from the same country are equal.

$H_0 : \mu_1 = \mu_2 = \mu_3 = ... = \mu_n$

Where $\mu_i$ represents the mean price of applications from country $i$, and $n$ is the total number of countries.

**Alternative Hypothesis (H1):** The mean prices of applications from the same country are not equal (at least one pair of means is different).

$H1 :\text{At least one pair of } \mu_i \text{ and } \mu_j \text{ is different, where } i \neq j$.

**Statistical Test:** 

I conducted the Kruskal-Wallis test for each country to determine if there are significant differences in prices between applications within the same country. The Kruskal-Wallis test is a non-parametric test used to compare more than two independent groups when the outcome variable is not normally distributed.

In [225]:
# Create a new column 'OS' to store the first word from 'osAndVersion'
df['OS'] = df['osAndVersion'].str.split('-').str[0]
df

,id,eventTimestamp,unitDisplayType,brandName,bundleId,countryCode,deviceId,osAndVersion,bidFloorPrice,sentPrice,OS
0,1,1657758857892,rewarded,Generic,com.loop.match3d,US,6a0a94554cf,Android-4.0,0.05,0.06,Android
1,2,1657210707978,interstitial,Generic,com.loop.match3d,ZA,6a0b0e59f45,Android-4.0,0.01,0.16,Android
2,3,1657392939412,interstitial,Motorola,com.YayySAL.DodgeAgent,BR,6a0fa820c46,Android-10.0,0.05,0.06,Android
3,4,1657386816882,interstitial,Huawei,com.tintash.nailsalon,IQ,6a142bdbea2,Android-9.0,0.01,0.05,Android
4,5,1657211600823,banner,Generic,com.tilegarden.match3,US,6a16943a771,Android-4.0,0.01,0.03,Android
...,...,...,...,...,...,...,...,...,...,...,...
41579,41580,1657731605063,banner,Generic,com.tilegarden.match3,NZ,80edf5837e8,Android-4.0,0.01,0.02,Android
41580,41581,1657657131101,interstitial,Apple,1436213906,VN,80ee6a5eebf,iOS-13.6,0.01,0.90,iOS
41581,41582,1657156070854,rewarded,Generic,com.loop.match3d,US,80efe531afc,Android-4.0,0.05,0.69,Android
41582,41583,1657659603568,banner,Apple,1502447854,US,80f02dd397f,iOS-15.0,0.01,0.10,iOS


In [246]:
# Group by 'countryCode' and aggregate 'sentPrice' into a list
df_grouped_country = df.groupby(['bundleId','countryCode'])['sentPrice'].agg(list).reset_index()
df_grouped_country

,bundleId,countryCode,sentPrice
0,1436213906,AE,"[0.02, 0.02, 33.03, 8.03, 0.02, 0.02, 4.45, 0...."
1,1436213906,AF,[0.02]
2,1436213906,AM,"[0.72, 0.7, 0.05, 0.55]"
3,1436213906,AR,[0.13]
4,1436213906,AT,"[0.02, 0.02, 0.02]"
...,...,...,...
1384,se.ace.fishinc,VE,"[0.72, 0.02, 0.05, 0.13, 0.26, 0.03, 0.03, 0.0..."
1385,se.ace.fishinc,VN,"[0.16, 0.88, 0.81, 0.96, 2.36, 0.02, 0.66, 0.3..."
1386,se.ace.fishinc,XK,[0.2]
1387,se.ace.fishinc,ZA,"[5.22, 0.38, 0.33, 0.26, 1.17, 0.54, 1.31, 0.6..."


In [257]:
# Perform the Kruskal-Wallis test for each countryCode (exclude single observations)
results = {}
for country, group in df_grouped_country.groupby('countryCode'):
    if len(group) > 1:  # Exclude groups with only one observation
        prices = [price for sublist in group['sentPrice'] for price in sublist]
        results[country] = kruskal(*group['sentPrice'])

# Create a DataFrame from the results
df_results_country = pd.DataFrame(index=results.keys(), columns=['H-statistic', 'p-value'])
for country, result in results.items():
    df_results_country.loc[country, 'H-statistic'] = result.statistic
    df_results_country.loc[country, 'p-value'] = result.pvalue

# Check if applications with the same countryCode have the same sentPrice based on the Kruskal-Wallis test
df_results_country['sentPrice_Status'] = df_results_country['p-value'].apply(lambda p: 'reject' if p < 0.05 else 'fail')
df_results_country

,H-statistic,p-value,sentPrice_Status
AE,10.757811,0.631098,fail
AF,5.5,0.357946,fail
AL,6.021778,0.537209,fail
AM,11.688409,0.231451,fail
AO,5.218182,0.265634,fail
...,...,...,...
UZ,35.160804,0.000056,reject
VE,23.783288,0.001244,reject
VN,203.598218,0.0,reject
ZA,43.050215,0.000011,reject


In [259]:
# Apply the function to each row and count the occurrences
frequency_counts = df_results_country['sentPrice_Status'].value_counts()

# Print the frequency of reject and fail outcomes
print("Frequency of Hypothesis Test Results:")
print(frequency_counts)

# Calculate the percentage of 'fail' entries and 'reject' enteries
percentage_reject = (frequency_counts.get('fail', 0) / frequency_counts.sum()) * 100
percentage_fail = (frequency_counts.get('fail', 0) / frequency_counts.sum()) * 100
print(f"Percentage of 'fail': {round(percentage_fail,2)}%")
print(f"Percentage of 'reject': {100-round(percentage_fail,2)}%")

# Determine the overall conclusion based on the majority outcome
if frequency_counts.get('reject', 0) > frequency_counts.get('fail', 0):
    print("\nThe hypothesis is consistent across the dataset. There is a significant difference in prices between apps with similar countryCode.")
else:
    print("\nThe hypothesis is not consistent across the dataset. There is no significant difference in prices between apps with similar countryCode.")

Frequency of Hypothesis Test Results:
sentPrice_Status
fail      84
reject    61
Name: count, dtype: int64
Percentage of 'fail': 57.93%
Percentage of 'reject': 42.07%

The hypothesis is not consistent across the dataset. There is no significant difference in prices between apps with similar countryCode.


**Results Interpretation:**
*  If the p-value from the Kruskal-Wallis test is less than the chosen significance level (e.g., 0.05), we **reject** the null hypothesis and conclude that there are significant differences in prices between applications within the same country.
*  If the p-value is greater than the significance level, we **fail** to reject the null hypothesis, indicating that there are no significant differences in prices between applications within the same country.ry.

**Conclusions:**

Based on the analysis, the majority of hypothesis tests (57.93%) failed to reject the null hypothesis, indicating that there is no significant difference in prices between applications with similar countryCode. In contrast, 42.07% of the hypothesis tests rejected the null hypothesis, suggesting that there are significant differences in prices between apps with similar countryCode. Therefore, the hypothesis is not consistent across the dataset, and the assumption of similar prices for apps with the same countryCode is not supported by the statistical analysis.

`OS`

**Null hypothesis (H0):** The mean prices of applications from the same operation system are equal.

$H_0 : \mu_1 = \mu_2 = \mu_3 = ... = \mu_n$

Where $\mu_i$ represents the mean price of applications from operation system $i$, and $n$ is the total number of countries.

**Alternative Hypothesis (H1):**  The mean prices of applications from the same operation system are not equal (at least one pair of means is different).

$H_1: \text{At least one pair of } \mu_i \text{ and } \mu_j \text{ is different, where } i \neq j$

**Statistical Test:** 

I conducted the Kruskal-Wallis test for each operation system to determine if there are significant differences in prices between applications within the same operation system. The Kruskal-Wallis test is a non-parametric test used to compare more than two independent groups when the outcome variable is not normally distributed.

In [242]:
# Group by 'bundleId' and 'OS' and aggregate 'sentPrice' into a list
df_grouped_os = df.groupby(['bundleId', 'OS'])['sentPrice'].agg(list).reset_index()
df_grouped_os

,bundleId,OS,sentPrice
0,1436213906,iOS,"[5.39, 6.69, 1.94, 0.52, 6.91, 0.02, 2.07, 0.0..."
1,1502447854,iOS,"[0.86, 0.07, 0.01, 0.15, 0.12, 4.58, 0.67, 0.0..."
2,1523081624,iOS,"[15.25, 0.02, 0.02, 0.02, 0.02, 0.2, 9.77, 0.0..."
3,1529614832,iOS,"[0.02, 0.18, 0.15, 0.16, 0.02, 0.02, 0.06, 0.5..."
4,1542256628,iOS,"[0.38, 0.25, 0.12, 0.07, 0.35, 0.35, 0.11, 0.1..."
5,1569586264,iOS,"[7.81, 29.81, 1.23, 9.16, 42.86, 7.15, 9.08, 8..."
6,1579489488,iOS,"[0.07, 0.14, 0.53, 0.06, 0.06, 0.06, 0.05, 0.1..."
7,1582745578,iOS,"[0.05, 1.42, 0.14, 0.44, 1.0, 0.05, 0.1, 0.06,..."
8,1586795332,iOS,"[0.14, 0.14, 0.46, 1.78, 0.07, 0.06, 14.59, 1...."
9,com.AppIdeas.LevelUpRunner,Android,"[0.19, 0.47, 0.07, 0.05, 0.07, 0.05, 0.08, 0.0..."


In [245]:
# Perform the Kruskal-Wallis test for each OS
results = {}
for os, group in df_grouped_os.groupby('OS'):
    prices = [price for sublist in group['sentPrice'] for price in sublist]
    results[os] = kruskal(*group['sentPrice'])

# Display the Kruskal-Wallis test results
for os, result in results.items():
    print(f'OS: {os}')
    print(f'Kruskal-Wallis H-statistic: {result.statistic}')
    print(f'Kruskal-Wallis p-value: {result.pvalue}')
    print('')

# Check if applications with the same OS have the same sentPrice based on the Kruskal-Wallis test
for os, result in results.items():
    if result.pvalue < 0.05:
        print(f'Applications with OS {os} have significantly different sentPrice.')
    else:
        print(f'Applications with OS {os} have similar sentPrice.')

OS: Android
Kruskal-Wallis H-statistic: 2202.1427310705085
Kruskal-Wallis p-value: 0.0

OS: iOS
Kruskal-Wallis H-statistic: 567.0597393085845
Kruskal-Wallis p-value: 2.81054555917353e-117

Applications with OS Android have significantly different sentPrice.
Applications with OS iOS have significantly different sentPrice.


**Results Interpretation:**
*  If the p-value from the Kruskal-Wallis test is less than the chosen significance level (e.g., 0.05), we **reject** the null hypothesis and conclude that there are significant differences in prices between applications within the same operation system.
*  If the p-value is greater than the significance level, we **fail** to reject the null hypothesis, indicating that there are no significant differences in prices between applications within the same operation system.

**Conclusions:**

Based on the analysis, the Kruskal-Wallis test results show that there are significant differences in sentPrice between applications with different operation systems (OS). Specifically:
* For applications with OS Android, the Kruskal-Wallis H-statistic is $2202.14$, and the p-value is $0.0$, indicating a significant difference in sentPrice.
* For applications with OS iOS, the Kruskal-Wallis H-statistic is $567.06$, and the p-value is approximately $2.81e-117$, also indicating a significant difference in sentPrice.

Therefore, the hypothesis that applications with the same OS have the same sentPrice is rejected based on the Kruskal-Wallis test results, ,and the assumption of similar prices for apps with the same OS is not supported by the statistical analysis.